# **StrongBlock Capital Gains Tax Estimator**

Well shams whats the story, doing a bit of the tax estimation are ya? Well your in the right place!

Please excuse the mess, didnt have time or effort to code a front-end, so just ignore the code and utilise the outputs. Follow along if you like and learn a bit of python. Will provide explanations as we go, so if you think theres any features worth adding give me a shout and I'll see if i can add them!





In an effort to make this all as accesible as possible we will be using google colab to execute our python cells. Colaboratory, or "Colab" for short, allows you to write and execute Python in your browser. Tis prety fucking handy if i do say so meself.

Were going to use a few Python data science libraries I'm familiar with through work - mainly NumPy for computation, Pandas for working with CSVs and Matplotlib for generating plots (still not sure if this is useful). We wil import them below - no need to do or change anything - everything is set up to go!

To start the estimator, go to the runtime tab up top and select tun all.


In [ ]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt


# **Input Required Here**


We will be getting the data for all transactions from **https://app.zerion.io/**. You will need to go to the website and connect your wallet. Once connected change your currency to Euro (EUR) using the drop-down menu in the top left of the page. Then select history from the tabs descending on the right hand of the page. Finally select download CSV to generate a CSV file of all transactions.

When thats done you can upload the CSV to colab using the button below.

In [ ]:
# # Generate upload button
# from google.colab import files
# import io


# zerion_csv = files.upload()

# # Grab the file name from the first dictionary key
# zerion_csv_file_name = list(zerion_csv.keys())[0]

Here I inport the csv and do some adjustments. To make it easier to work with in python. I replace the spaces in the column headers with underscores (_'s) and fill in any NA values in the buy and sell currancy rows to avoid any issues when preforming nemerical or search operations etc.

In [ ]:
import pandas as pd

# Create panda dataframe from the zerion CSV
df = pd.read_csv ("/Users/eamonmcandrew/Desktop/Portfolio_tools/StrongBlock/0x3598f618d45e02b92d9b4d8129d45dbd1b0beb5d transactions 01_20_2022 07_47 from Zerion.csv")

# Some data processing to make certain value more accesible in python
df.columns = df.columns.str.replace(' ', '_')
df['Buy_Currency'] = df['Buy_Currency'].fillna("_")
df['Sell_Currency'] = df['Sell_Currency'].fillna("_")




# Date selection 
The code cell below allows you to select the date range for the tax period you wish to calculate for. Just change the dates below and use run all again to restart the whole notebook and run. Best to do this everythime to keep everything accurate and running as I intended. Good thing is this can be done really quickly.

In [ ]:
df = df[df.Date.between("01/01/2021", "12/13/2021")]


Here i'm going to calculate all the fees incurred when disposing (selling) $STRONG. I have done this through Uniswap and Cowswap - so essentially I understand there are two tax deductable fees here the ETH network fee and the relative exchange fees.

In [ ]:
# Create a dataframe of all times strong is sold 
Strong_swap_to_other_coin = df[df['Sell_Currency'].str.match("STRONG")]

# Sum the eth fees associated with each transaction.
Strong_swap_ETH_fees = Strong_swap_to_other_coin.Fee_Fiat_Amount.sum()

print("Strong Swap ethereum network Fees", Strong_swap_ETH_fees)

# Cowswap utilises a different mechanism to swap and the fees are paid using the currency you are converting to
# Therefore the Eth network fees for these transactions are not available in the CSV. But we can get the exchange
# fees by taking the fiat value of the coin being sold and subtracting it from the the fiat value of the coin
# recieved. The difference is the Cowswap exchange fees.
Strong_swap_to_other_coin_cowswap = Strong_swap_to_other_coin[Strong_swap_to_other_coin['Fee_Amount'].isnull()]
Strong_swap_to_other_coin_cowswap = Strong_swap_to_other_coin_cowswap[["Buy_Fiat_Amount" , "Sell_Fiat_Amount", "Sell_Amount"]]
Strong_swap_to_other_coin_cowswap["Fees"] = Strong_swap_to_other_coin_cowswap["Sell_Fiat_Amount"].astype(float) - Strong_swap_to_other_coin_cowswap["Buy_Fiat_Amount"].astype(float)
Cowswap_fees = Strong_swap_to_other_coin_cowswap.Fees.sum()

print("Strong Conversion Fees  Cowswap", Cowswap_fees)



Next I will calculate the strongblock apporval fees. This is the Eth fees paid when you approve a protocol to use strong. Unfortunately there was no way for me to automate this easily.. you probably will not have too many of these other than approving strong for node creation and approving strong for sale. If you want to use this feature then you will need to maunually copy the approval transactions from zerion and replace them with my ones below, just delete any extra TXs if you dont have as many (each tx is "number" followd by a comma). Mine came to very little so It might not be worth the effort for you.

In [ ]:



Approvals = df[df['Transaction_Type'].str.contains("Approval")]
Strong_approval_TXs = ["0xb18ddb38d9031fd800d649ab7e471e836427827fe5cef5dc901856d53898e97f", "0x7171326dc1141bdb64e7b4b0b8034ca0d2586a1dc80ba1ef5078772c6c0f8840", "0xf5357f7998306b6740c366946ee29d084928473bf5c2f9690c0f3a95e31fd11d", "0x5be511a6dc9e0d90c8b90b7009f7f6373f06b551a91c0b986f6292183c5a33af"]

Strong_approval_TXs[0]
Strong_approvals = []
for i in range(len(Strong_approval_TXs)):
  Strong_Approval = Approvals[Approvals['Tx_Hash'].str.contains(Strong_approval_TXs[i])]
  Strong_approvals.append(Strong_Approval)

Strong_approvals_df = pd.concat(Strong_approvals, axis=0, ignore_index=True)





In [ ]:
Strong_approvals_df

Next I will calculate the cost of all $STRONG bought (assuming you only used strong bought to make nodes - makes compounding calculation eaiser).

In [ ]:
# Get total Incoming $STRONG transactions as a df
Strong_To_Wallet = df[df['Buy_Currency'].str.contains("STRONG")]


In [ ]:
# Get total incoming strong claims as a df, we can use the reciever address to distinguish these transactions.

Strong_Claims = Strong_To_Wallet[Strong_To_Wallet['Receiver'].str.contains("0xfbddadd80fe7bda00b901fbaf73803f2238ae655")]

#Get total incoming strong buys 
Strong_Buys = Strong_To_Wallet[Strong_To_Wallet['Receiver'].str.contains("0x7a250d5630b4cf539739df2c5dacb4c659f2488d")]
Strong_buy_total = Strong_Buys.Buy_Fiat_Amount.astype(float).sum()

Strong_buy_total

In [ ]:
Strong_Claims

In [ ]:
Strong_claims = Strong_Claims[["Date", "Sell_Fiat_Amount", "Fee_Fiat_Amount", "Buy_Fiat_Amount", "Tx_Hash", "Link"]]
Strong_claims_spent_total_column = Strong_claims["Sell_Fiat_Amount"].astype(float) + Strong_claims["Fee_Fiat_Amount"].astype(float)
Strong_claims["total_paid_claims"] = Strong_claims_spent_total_column
Strong_claims.set_index('Date', inplace=True)

Strong_claims 

In [ ]:
Strong_claims_display = Strong_claims
Strong_claims_display = Strong_claims_display.rename(columns={"Sell_Fiat_Amount": "Claim Fee", "Fee_Fiat_Amount": "ETH Network Fee Fiat", "Buy_Fiat_Amount": "Amount Claimed Fiat", "total_paid_claims": "Total Claim Fess Fiat", "Tx_Hash": "Transaction Hash", "Link": "Etherscan Link" })



In [ ]:
Strong_claims_display = Strong_claims_display[["Amount Claimed Fiat", "Total Claim Fess Fiat", "Claim Fee", "ETH Network Fee Fiat", "Transaction Hash", "Etherscan Link" ]]
Strong_claims_display

In [ ]:
Strong_claims.to_csv("Strong_claim_data.csv")

In [ ]:
Strong_claims


Strong_Claims_EUR_total = Strong_Claims.Buy_Fiat_Amount.astype(float).sum()


In [ ]:
Strong_leaving_Wallet = df[df['Sell_Currency'].str.contains("STRONG", na=False)]



In [ ]:
Node_creation_transactions = Strong_leaving_Wallet[Strong_leaving_Wallet['Sell_Currency'].str.contains("ETH\nSTRONG", na=False)]
Total_nodes_created = Node_creation_transactions.shape[0]



In [ ]:
Node_creation_transactions

In [ ]:
# Node_creation_transactions =  Node_creation_transactions[["Date", "Sell_Fiat_Amount", "Fee_Fiat_Amount", "Buy_Fiat_Amount", "Tx_Hash", "Link"]]
# Node_creation_transactions.set_index('Date', inplace=True)


In [ ]:
Node_creation_transactions


Node_creation_gas_fees_total = Node_creation_transactions.Fee_Fiat_Amount.sum()

Node_creation_gas_fees_total

In [ ]:
Node_creation_transactions_display = Node_creation_transactions
new = Node_creation_transactions_display[['Sell_Fiat_Amount']]
new


In [ ]:
# new data frame with split value columns
new = new["Sell_Fiat_Amount"].str.split("\n", n = 1, expand = True)
Node_creation_transactions_display["Creation Fee ETH Fiat"] = new[0]
Node_creation_transactions_display["Creation Fee STRONG Fiat"] = new[1]

In [ ]:
Node_creation_transactions_display = Node_creation_transactions_display[["Date", "Creation Fee ETH Fiat", "Creation Fee STRONG Fiat","Fee_Fiat_Amount", "Tx_Hash", "Link"]]
Node_creation_transactions_display = Node_creation_transactions_display.rename(columns={"Sell_Fiat_Amount": "Claim Fee", "Fee_Fiat_Amount": "ETH Network Fee Fiat", "Buy_Fiat_Amount": "Amount Claimed Fiat", "total_paid_claims": "Total Claim Fess Fiat", "Tx_Hash": "Transaction Hash", "Link": "Etherscan Link" })
Node_creation_transactions_display  = Node_creation_transactions_display.rename(columns={"Sell_Fiat_Amount": "Claim Fee", "Fee_Fiat_Amount": "ETH Network Fee Fiat", "Buy_Fiat_Amount": "Amount Claimed Fiat", "total_paid_claims": "Total Claim Fess Fiat", "Tx_Hash": "Transaction Hash", "Link": "Etherscan Link" })


In [ ]:
Node_creation_transactions_display

In [ ]:


Creation_dates = Node_creation_transactions[['Date', 'Sell_Fiat_Amount']]

def sum_node_creation_cost(dual_cost):
  value = str(dual_cost).split("\n")[0] , str(dual_cost).split("\n")[1]
  value = float(value[0]) + float(value[1])
  return value


Creation_dates['Sell_Fiat_Amount'] = [ sum_node_creation_cost(x) for x in Creation_dates['Sell_Fiat_Amount'] ] 

Creation_dates

In [ ]:
Creation_dates.to_csv("node_creation_dates.csv")

In [ ]:
Creation_dates['Dates'] = [datetime.strptime(date, '%m/%d/%Y').date() for date in Creation_dates['Date']]


In [ ]:


Fee_list = list(Node_creation_transactions.Sell_Fiat_Amount)




Eth_fees_EUR = []
Strong_fees_EUR = []

for i in range(len(Fee_list)):
   Eth_fee = Fee_list[i].split("\n")[0]
   Eth_fee = float(Eth_fee)
   Eth_fees_EUR.append(Eth_fee)
   STRONG_fee = Fee_list[i].split("\n")[1]
   STRONG_fee = float(STRONG_fee)
   Strong_fees_EUR.append(STRONG_fee)



In [ ]:
Total_ETH_creation_fees = sum(Eth_fees_EUR)
Total_STRONG_creation_fees = sum(Strong_fees_EUR)


In [ ]:
Strong_Monthly_fees = df[df['Receiver'].str.contains("0xfbddadd80fe7bda00b901fbaf73803f2238ae655")]
Strong_Monthly_fees = Strong_Monthly_fees[Strong_Monthly_fees['Accounting_Type'].str.contains("Spend")]
Strong_Monthly_fees = Strong_Monthly_fees[Strong_Monthly_fees['Sell_Currency'] == "ETH"]




In [ ]:
Monthly_Fee_list = list(Strong_Monthly_fees.Sell_Fiat_Amount)
Monthly_Fee_Gas_list = list(Strong_Monthly_fees.Fee_Fiat_Amount)

Monthly_Fees_EUR = []
Monthly_Fee_Gas_Fees_EUR = []

for i in range(len(Monthly_Fee_list)):
  Monthly_Fees_EUR.append(float(Monthly_Fee_list[i]))
  Monthly_Fee_Gas_Fees_EUR.append(float(Monthly_Fee_Gas_list[i])) 
   
Total_monthly_fees_EUR = sum(Monthly_Fees_EUR)

Total_monthly_gas_fees_EUR = sum(Monthly_Fee_Gas_Fees_EUR)

   
Total_monthly_fees_EUR



Right, this one is a bolox to calculate as i dont think zerion records this type of transaction. Essentially I want to calculate the fees for each transfer of ETH or USD-based stablecoin to Coinbase, which is my fiat off-ramp at present. Essentially I need to calulate the EUR value of the ethereum gas fee for each of these transactions, simple eh? Not really Coinbase generates a new send address every time and zerion doesnt record these transactions.

In [ ]:
# Etherscan_CSV = files.upload()
# # Grab the file name from the first dictionary key
# Etherscan_CSV  = list(Etherscan_CSV.keys())[0]
# Etherscan_df = pd.read_csv(Etherscan_CSV, index_col=False)
# Etherscan_df





In [ ]:
# Etherscan_df = pd.read_csv(Etherscan_CSV, index_col=False)

In [ ]:
# Etherscan_df

In [ ]:
# Coinbase_wallets = files.upload()
# Coinbase_wallets = list(Coinbase_wallets.keys())[0]


In [ ]:
# with open(Coinbase_wallets) as file:
#     lines = file.readlines()
#     Coinbase_wallet_list = [line.rstrip() for line in lines]

In [ ]:
# def String_match_list(df,col,searchfor):
#   df["TrueFalse"] = df[str(col)].apply(lambda x: True if any(i in x for i in searchfor) else False)
#   df = df[df["TrueFalse"]]
#   return df

In [ ]:
# to_cb = String_match_list(Etherscan_df,"To",Coinbase_wallet_list )

# to_cb


In [ ]:
# Fees for all transactions for incoming $STRONG in EUR fiat value. This includes purchases and claims.

Total_fees_IN = Strong_To_Wallet.Fee_Fiat_Amount.sum()
print("Total Incoming $STRONG Fees €",round(Total_fees_IN, 2))

Strong_Buy_fees = Strong_Buys.Fee_Fiat_Amount.sum()
print("Total Incoming $STRONG Purchase Fees €",round(Strong_Buy_fees, 2))

Strong_Claim_fees = Strong_Claims.Fee_Fiat_Amount.sum()
print("Total Incoming $STRONG Claim Fees €", round(Strong_Claim_fees,2))

print(" ")

print("Total $STRONG Claimed by EUR value at time of claim €", round(Strong_Claims_EUR_total,2))


# print("Total $STRONG Claimed by $STRONG value at time of claim ",Strong_claims_STRONG_total)

print(" ")
print("Total Nodes created (All Time)", round(Total_nodes_created,2), "Strong Nodes")
print("Total $ETH creation Fees in EUR €", round(Total_ETH_creation_fees,2))
print("Total $STRONG creation Fees in EUR €", round(Total_STRONG_creation_fees,2))
print("Total Gas fees for node creation in EUR €", round(Node_creation_gas_fees_total,2))

print(" ")
print("Total monthly Node Fees EUR €", round(Total_monthly_fees_EUR,2))
print("Total gas paid in paying monthly fees €", round(Total_monthly_gas_fees_EUR,2) )

strong_compund_total = Total_STRONG_creation_fees - Strong_buy_total
tax_deductable_total = round(Total_nodes_created,2) + round(Total_ETH_creation_fees,2) +  round(strong_compund_total *0.66666,2) +  round(Node_creation_gas_fees_total,2) + round(Total_monthly_gas_fees_EUR,2)+round(Total_monthly_fees_EUR,2)+round(Strong_swap_ETH_fees,2)+round(Cowswap_fees,2)
print(" ")
print("Total tax deductable fees", tax_deductable_total)

payment_due = round(Strong_Claims_EUR_total,2) - tax_deductable_total
print("Total payment due", payment_due*0.333)